In [38]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [39]:
def load_dataset(csv_path):
    df = pd.read_csv(csv_path, sep= ';')
    required_columns = ['Azimuth', 'Altitude', 'Timestamp']

    if not all (col in df.columns for col in required_columns):
        raise ValueError("Required columns are not present in the DataFrame.")
    
    X = df[['Timestamp','Azimuth','Altitude']].values
    y = df['Azimuth'].values

    return X, y, df

In [40]:
csv_path = 'cleaned/cleaned_data_quartal_1_2020.csv'
X, y, df = load_dataset(csv_path)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state= 42)

In [42]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))

In [43]:
timesteps = 1

In [44]:
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

In [45]:
model = Sequential()
model.add(LSTM(units=1024, activation='relu', input_shape=(timesteps, X_train_scaled.shape[1]), return_sequences=True))
model.add(LSTM(units=512, activation= 'relu', return_sequences=True))
model.add(LSTM(units=256, activation= 'relu', return_sequences=True))
model.add(LSTM(units=128, activation= 'relu', return_sequences=True))
model.add(LSTM(units=64, activation= 'relu', return_sequences=True))
model.add(LSTM(units=32, activation= 'relu'))
model.add(Dense(units=1))

optimizer = Adam(learning_rate= 0.0001)
model.compile(optimizer=optimizer, loss = 'mse')

model.fit(X_train_reshaped, y_train_scaled, epochs= 500, batch_size = 256)

Epoch 1/500
41/41 [==============================] - 7s 37ms/step - loss: 0.2875
Epoch 2/500
41/41 [==============================] - 1s 36ms/step - loss: 0.2788
Epoch 3/500
41/41 [==============================] - 1s 35ms/step - loss: 0.2685
Epoch 4/500
41/41 [==============================] - 1s 35ms/step - loss: 0.2498
Epoch 5/500
41/41 [==============================] - 1s 35ms/step - loss: 0.1342
Epoch 6/500
41/41 [==============================] - 1s 36ms/step - loss: 0.0326
Epoch 7/500
41/41 [==============================] - 1s 34ms/step - loss: 0.0236
Epoch 8/500
41/41 [==============================] - 1s 36ms/step - loss: 0.0156
Epoch 9/500
41/41 [==============================] - 1s 35ms/step - loss: 0.0080
Epoch 10/500
41/41 [==============================] - 1s 36ms/step - loss: 0.0026
Epoch 11/500
41/41 [==============================] - 1s 36ms/step - loss: 7.9981e-04
Epoch 12/500
41/41 [==============================] - 1s 35ms/step - loss: 5.2698e-04
Epoch 13/500
41/4

In [ ]:
predicted_values_scaled = model.predict(X_test_reshaped)
predicted_values = scaler_y.inverse_transform(predicted_values_scaled)
mse = mean_squared_error(y_test_scaled, predicted_values_scaled)
r2 = r2_score(y_test_scaled, predicted_values_scaled)
accuracy = model.evaluate(X_test_reshaped, y_test_scaled)
print(f"Mean Squared Error on Test Data: {mse}")
print(f"R-squared on Test Data: {r2}")
print(f"Accuracy on Test Data: {accuracy}")


82/82 [==============================] - 1s 7ms/step - loss: 4.3617e-07
Mean Squared Error on Test Data: 4.3616431806443825e-07
R-squared on Test Data: 0.9999896793164073
Accuracy on Test Data: 4.361651804174471e-07
